# Jetpilot - Data collection

In this notebook we'll collect training data the same as [DonkeyCar](https://www.donkeycar.com/). 
The training data save to dataset directory. Need gamepad controller is connected to Laptop with USB.

## Import module

Import required module for this notebook. MobileController module is own module. This module implements Jetbot steering control.


In [1]:
import os
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

In [ ]:
LOG_DIRECTORY = 'dataset'
CAMERA_WIDTH = 320
CAMERA_HEIGHT = 240

In [6]:
CAMERA_WIDTH = 320
CAMERA_HEIGHT = 240

camera = CSICamera(width=CAMERA_WIDTH, height=CAMERA_HEIGHT, capture_width=CAMERA_WIDTH, 
                   capture_height=CAMERA_HEIGHT, capture_fps=60)

camera.running = True

image = widgets.Image(format='jpeg', width=320, height=240)
camera_link = traitlets.dlink((camera,'value'), (image,'value'), transform=bgr8_to_jpeg)
display(image)

## UI Widget

We can check Gamepad value and Image. 

In [7]:
from IPython.display import display
import ipywidgets

log_button = ipywidgets.ToggleButton(value=False, description='enable logging')
display(log_button)


Directories not created becasue they already exist


## Set callback for collect the training data.

```sace_record``` is callback for training data. The method set to camera observer. This callback saving the image that contain speed and steering in file name to DATASET_DIR. When holding ```R``` button, this method recording training data. You can check number of training data with ```Number image text box```. If We plan learning on Jetbot, Number of training data less than 700. 

In [8]:
if not os.path.exists(LOG_DIRECTORY):
    os.mkdir(LOG_DIRECTORY)

count = 0

def execute(change):
    global count, LOG_DIRECTORY
    image = change['new']
    if log_button.value:
        image_path = os.path.join(LOG_DIRECTORY, '%012d.jpg' % count)
        cv2.imwrite(image_path, image)
        count += 1
    else:
        count = 0

execute({'new': camera.value})
camera.observe(execute, names='value')

## Cleanup
After collecting enough data. cleanup camera observer and stop all motor.

In [9]:
camera.unobserve_all()

If you leaning Autopilot on Cloud then run next cell.

In [10]:
import datetime
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q jetbot_imitiative_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}